# Previsões e séries temporais
Link: [https://github.com/alvarodiegomaia/time-series-presentation](https://github.com/alvarodiegomaia/time-series-presentation)



## Álvaro Diego Maia

- Físico pela Universidade de Brasília
- Mestre pela Universidade de São Paulo
- Doutor pela Universidade de São Paulo
- Pós-doutorados:
    - PUC Rio
    - Sorbonne Université - campus Marie et Pierre CURIE
    - Alcatel Labs Paris
    - Universidade de Brasília

linkedin: [linkedin.com/in/alvarodiego/](https://www.linkedin.com/in/alvarodiego/)

# Forecasting (previsões)

- Previsão do futuro;
- A partir de dados do passado ou atuais;
- Analisando tendências.

## Situações

- Decidir se será construída outra usina de energia
- Programar o tamanho de uma equipe de trabalho;
- Previsão do estoque de um inventário.

O tempo da previsão também pode mudar.


## Predibilidade de um evento

- Conhecemos os fatores?
- Há muitos dados disponíveis?
- O comportamento do passado é similar ao do presente?
- As previsões afetam o que estamos tentando prever?
- Quão distante é o horizonte que queremos prever?

## Vivemos cercados de previsões

**"I think there is a world market for maybe five computers"**
- *Thomas Watson, presidente da IBM, 1943*


**"Computers in the future may weigh no more than 1.5 tons"**
- *Popular Mechanics, 1949*


**"There is no reason anyone would want a computer in their home"**
- *Ken Olsen, presidente da DEC, 1977*

**"We're going to be opening relatively soon... I would love to have the country opened up and just raring to go by Easter"**
- *Donald Trump, Fevereiro 2020*

## Qual a dificuldade dessas predições

- Horário do nascer do sol no próximo ano
- Calendário da próxima aparição do cometa Halley
- Temperatura máxima amanhã
- Demanda de energia nos próximos 3 dias
- Preço de uma ação amanhã
- Cambio do dólar na próxima semana
- Preço de uma ação daqui há seis meses

## Caso da demanda de transprotes
1. Conhecemos os fatores?
2. Há muitos dados disponíveis?
3. O comportamento do passado é similar ao do presente?
4. As previsões afetam o que estamos tentando prever?
5. Quão distante é o horizonte que queremos prever?


# Importando bibliotecas

In [252]:
import numpy as np
import pandas as pd
 
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import dates as mdates
from matplotlib.ticker import EngFormatter
import matplotlib.ticker as mticker
from matplotlib.axis import Axis
 
import seaborn as sns
 
import plotly
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
 
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf, pacf, adfuller, kpss
from statsmodels.tsa.seasonal import STL
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
 
import sktime
from sktime import datasets
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.fbprophet import Prophet
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.ets import AutoETS
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError

from tqdm import tqdm

In [253]:
print('Versions')
print(f'numpy: {np.__version__}')
print(f'pandas: {pd.__version__}')
print(f'matplotlib: {matplotlib.__version__}')
print(f'seaborn: {sns.__version__}')
print(f'plotly: {plotly.__version__}')
print(f'statsmodels: {statsmodels.__version__}')
print(f'sktime: {sktime.__version__}')

Versions
numpy: 1.23.5
pandas: 1.5.3
matplotlib: 3.7.1
seaborn: 0.12.2
plotly: 5.14.1
statsmodels: 0.14.0
sktime: 0.18.1


## Métodos

In [254]:
def perform_adf_test(serie, maxlag=20):
    resultado = adfuller(
    serie,
    maxlag=maxlag,
    regression='ct',
    autolag='AIC',
    store=False,
    regresults=False
    )
 
    # print(f'ADF estatística: {resultado[0]}')
    # print(f'p-valor: {resultado[1]}')
 
    return (resultado[0], resultado[1], resultado[4]['1%'])
 

In [255]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
 
def analise_serie(serie, titulo, maxlag=20):
    """
    Traça os gráficos de analise de uma série temporal:
        - Estacionaridade pelo teste aumentado de Dickey-Fuller;
        - Cálculo de autocorrelação;
        - Cálculo de autocorrelação parcial.
 
    Args:
        serie (pd.Series): Série temporal a ser analisada.
        titulo (string): Título da série.
    """
 
 
    # Calcula ACF
    acf_array = acf(serie.dropna(), alpha=0.05)
    acf_lower_y = acf_array[1][:,0] - acf_array[0]
    acf_upper_y = acf_array[1][:,1] - acf_array[0]
 
    # Calcula PACF
    pacf_array = pacf(serie.dropna(), alpha=0.05)
    pacf_lower_y = pacf_array[1][:,0] - pacf_array[0]
    pacf_upper_y = pacf_array[1][:,1] - pacf_array[0]
 
    # Realiza o teste aumentado de Dickey-Fuller
    adf, pvalor, crit = perform_adf_test(serie, maxlag)
 
    # Cria os subplots com os títulos exibindo o resultado do Dickey-Fuller
    fig = make_subplots(
        rows=2,
        cols=2,
        specs=[
            [{"colspan": 2}, None],
            [{}, {}]
        ],
        subplot_titles=(
            f'Dickey-Fuller: {adf:.5f}\t crit 1%: {crit:.5f}\t p-valor: {pvalor:.5f}',
            'Auto-correlação',
            'Auto-correlação parcial'
        )
    )
 
    # Gráfico da série observada
    fig.add_trace(
        go.Scatter(
            x=serie.index,
            y=serie.values,
            line_color='#1f77b4'
        ),
        row=1,
        col=1
    )
 
 
    # Gráfico da autocorrelação
    [
        # Linhas verticais
        fig.add_scatter(
            x=(x, x),
            y=(0, acf_array[0][x]),
            mode='lines',
            line_color='#3f3f3f',
            row=2,
            col=1
        ) for x in range(len(acf_array[0]))
    ]
 
    fig.add_scatter(
        # Marcadores
        x=np.arange(len(acf_array[0])),
        y=acf_array[0],
        mode='markers',
        marker_color='#1f77b4',
        marker_size=12,
        row=2,
        col=1
    )
 
    fig.add_scatter(
        # Intervalo de confiança superior
        x=np.arange(len(acf_upper_y)),
        y=acf_upper_y,
        mode='lines',
        line_color='rgba(255,255,255,0)',
        row=2,
        col=1
    )
 
    fig.add_scatter(
        # Intervalo de confiança inferior
        x=np.arange(len(acf_lower_y)),
        y=acf_lower_y,
        mode='lines',
        fillcolor='rgba(32, 146, 230,0.3)',
        fill='tonexty',
        line_color='rgba(255,255,255,0)',
        row=2,
        col=1
    )
 
 
    # Gráfico da autocorrelação parcial
    [
        # Linhas verticais
        fig.add_scatter(
            x=(x, x),
            y=(0, pacf_array[0][x]),
            mode='lines',
            line_color='#3f3f3f',
            row=2,
            col=2
        ) for x in range(len(pacf_array[0]))
    ]
 
    fig.add_scatter(
        # Marcadores
        x=np.arange(len(pacf_array[0])),
        y=pacf_array[0],
        mode='markers',
        marker_color='#1f77b4',
        marker_size=12,
        row=2,
        col=2
    )
 
    fig.add_scatter(
        # Intervalo de confiança superior
        x=np.arange(len(pacf_upper_y)),
        y=pacf_upper_y,
        mode='lines',
        line_color='rgba(255,255,255,0)',
        row=2,
        col=2
    )
 
    fig.add_scatter(
        # Intervalo de confiança inferior
        x=np.arange(len(pacf_lower_y)),
        y=pacf_lower_y,
        mode='lines',
        fillcolor='rgba(32, 146, 230,0.3)',
        fill='tonexty',
        line_color='rgba(255,255,255,0)',
        row=2,
        col=2
    )
 
    fig.update_layout(
        showlegend=False,
        title_text=f'Análise da série {titulo}',
        height=300,
        width=900,
        margin={
            'l': 20,
            'r': 20,
            't': 60,
            'b': 20
        },
    )
    fig.show()

In [256]:
def series_decomposition(series, title, periodo=None):
    """
    Método para decompor séries temporais.
    Exibe os gráficos de suas componentes juntamente com o observável da série.
 
    Args:
        serie (pd.Series): Série temporal a ser analisada.
        periodo (int): Periodicidade da sequência.
    """
 
    decomp = STL(series, period=periodo).fit()
 
    df_decomp = pd.DataFrame(
        {
            'Observado' : decomp.observed,
            'Tendência e cíclico' : decomp.trend,
            'Sazonalidade' : decomp.seasonal,
            'Ruído' : decomp.resid
        }
    )
 
    df_decomp = pd.melt(df_decomp.reset_index(), id_vars='Período', var_name='Componente', value_name='Qtd passageiros').dropna()
 
    fig = px.line(
        df_decomp.reset_index(),
        x='Período',
        y='Qtd passageiros',
        facet_row='Componente',
        labels={
            'value': 'Média da carga total',
            'variable': 'Trecho',
            'Data_hora': 'Data/hora'
        },
    )
    fig.update_yaxes(
        showticklabels=True,
        matches=None
    )
    fig.update_xaxes(
        matches=None
    )
    fig.update_layout(
        barmode="group",
        showlegend=True,
        title={
            "text": f'Componentes da série {title}',
            "x": 0.5,
            "xanchor": "center",
            "yanchor": "top",
        },
        height=600,
        width=900,
        margin={
            'l': 20,
            'r': 20,
            't': 60,
            'b': 20
        },
    )
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.update_yaxes(visible=False, showticklabels=False)
    fig.show()

In [257]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
 
def plot_adf(serie, titulo, maxlag=20):
    """
    Traça os gráficos de analise de uma série temporal:
        - Estacionaridade pelo teste aumentado de Dickey-Fuller;
        - Cálculo de autocorrelação;
        - Cálculo de autocorrelação parcial.
 
    Args:
        serie (pd.Series): Série temporal a ser analisada.
        titulo (string): Título da série.
    """
 
 
    # Calcula ACF
    acf_array = acf(serie.dropna(), alpha=0.05)
    acf_lower_y = acf_array[1][:,0] - acf_array[0]
    acf_upper_y = acf_array[1][:,1] - acf_array[0]
 
    # Calcula PACF
    pacf_array = pacf(serie.dropna(), alpha=0.05)
    pacf_lower_y = pacf_array[1][:,0] - pacf_array[0]
    pacf_upper_y = pacf_array[1][:,1] - pacf_array[0]
 
    # Realiza o teste aumentado de Dickey-Fuller
    adf, pvalor, crit = perform_adf_test(serie, maxlag)
 
    # Cria os subplots com os títulos exibindo o resultado do Dickey-Fuller
    fig = make_subplots(
        rows=1,
        cols=2,
        specs=[
            [{"colspan": 2}, None]
        ],
        subplot_titles=(
            f'Dickey-Fuller: {adf:.5f}\t crit 1%: {crit:.5f}\t p-valor: {pvalor:.5f}',
            'Autocorrelação',
            'Autocorrelação parcial'
        )
    )
 
    # Gráfico da série observada
    fig.add_trace(
        go.Scatter(
            x=serie.index,
            y=serie.values,
            line_color='#1f77b4'
        ),
        row=1,
        col=1
    )
 
 
    fig.update_layout(
        showlegend=False,
        #title_text=f'Análise da série {titulo}',
        height=300,
        width=900,
        margin={
            'l': 20,
            'r': 20,
            't': 40,
            'b': 20
        },
    )
    fig.show()

In [258]:
def get_slidewindows_steps(init_lenght, train_length, drop_lenght, test_lenght, step_lenght, n_steps):
    idx = [{}]
    for step in range(n_steps):
        train_init = init_lenght + step * step_lenght
        train_end = train_init + train_length
        train_points = list(range(train_init, train_end))
        
        drop_init = train_end
        drop_end = drop_init + drop_lenght
        drop_points = list(range(drop_init, drop_end))
        
        test_init = drop_end
        test_end = test_init + test_lenght
        test_points = list(range(test_init, test_end))
        
        idx.append(
            {
                'train': train_points,
                'drop': drop_points,
                'test': test_points,
            }
        )
        
    return (idx[1:])

In [259]:
def plot_previsao(serie, resultado, modelo, previsao):
    """
    Traça os gráficos da série temporal e as previsões

    Args:
        serie (pd.Series): Série temporal a ser analisada.
        resultado (pd.DataFrame): Pandas dataframe com os resultados dos modelos.
        modelo (string): Nome do modelo.
        previsao (string): Nome da previsão usada.
    """
    
    data_plot = result.query('model == @modelo & previsao == @previsao')
    
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=y.to_timestamp(freq='M').index,
            y=y.values,
            name='Série temporal'
        )
    )

    [fig.add_trace(
        go.Scatter(
            x=data_plot['y_pred'][i].to_timestamp(freq='M').index,
            y=data_plot['y_pred'][i].values,
        )
    ) for i in range(10)]

    layout = go.Layout(
        {
            'title': {
                'text': f'Modelo: {modelo}; Previsão: {previsao}',
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top',
            },
            'showlegend': False,
            'height':300,
            'width':900,
            'margin': {
                'l': 20,
                'r': 20,
                't': 40,
                'b': 20
            },
        }
    )

    fig.update_layout(layout)

    fig.show()

In [260]:
def plot_desempenho(resultado, modelos, previsoes):
    """
    Traça gráficos do desempenhos dos modelos treinados.

    Args:
        resultado (pd.DataFrame): Pandas dataframe com os resultados dos modelos.
        modelos (list): Lista de strings com os nomes dos modelos.
        previsoes (list): Lista de strings com os nomes das previsões usadas.
    """


    plot_data = result.query('model == @modelos & previsao == @previsoes')
    
    fig = px.box(
        plot_data,
        y='mape',
        x='model',
        color='previsao',
        labels={
            'mape': 'Erro percentual médio absoluto',
            'previsao': 'Previsão',
            'model': 'Modelo'
        }
    )
    fig.update_yaxes(
        showticklabels=True,
        range=[0, 0.25]
    )
    fig.update_layout(
        showlegend=True,
        title={
            'text': 'Desempenho dos modelos',
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
        },
        height=400,
        width=800,
        margin={
            'l': 20,
            'r': 20,
            't': 60,
            'b': 20
        },
    )
    fig.layout.yaxis.tickformat = ',.0%'
    fig.show()

# Datasets

## Série temporal dos passagerios aéreos

Esse dataset contém uma tabela com o número de passagerios de companhias aéreas por mês.

In [261]:
df_air = pd.DataFrame(datasets.load_airline()).reset_index().rename(
    columns={
        'Period': 'Período',
        'Number of airline passengers': 'Qtd passageiros'
    }
)
df_air.head()

,Período,Qtd passageiros
0,1949-01,112.0
1,1949-02,118.0
2,1949-03,132.0
3,1949-04,129.0
4,1949-05,121.0


Convertendo a coluna `Período` para o tipo de dados de data e hora.

In [262]:
df_air['Período'] = pd.to_datetime(df_air['Período'].astype('str'))

In [263]:
df_air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Período          144 non-null    datetime64[ns]
 1   Qtd passageiros  144 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.4 KB


## Visualizando a série temporal

In [264]:
df_air.head()

,Período,Qtd passageiros
0,1949-01-01,112.0
1,1949-02-01,118.0
2,1949-03-01,132.0
3,1949-04-01,129.0
4,1949-05-01,121.0


In [265]:
print('Duração da série:')
print(f'{df_air["Período"].min()} até {df_air["Período"].max()}')
print(f'{df_air.shape[0]} registros')

Duração da série:
1949-01-01 00:00:00 até 1960-12-01 00:00:00
144 registros


## Visualizando a série temporal

In [266]:
plotly.offline.init_notebook_mode(connected=True)

fig = px.line(
    df_air,
    x='Período',
    y='Qtd passageiros',
    hover_data='Qtd passageiros',
    #labels={"Total_kg": "Carga total de passageiros (kg)", "dt_hour": "Hora do dia"},
    height=400,
    width=900,
)
#fig.update_yaxes(showticklabels=True)
fig.update_layout(
    barmode="group",
    showlegend=True,
    title={
        #"text": "Movimentação por hora do dia",
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    },
    height=300,
    width=900,
    margin={
        'l': 20,
        'r': 20,
        't': 40,
        'b': 20
    },
)
fig.show()

# Análise exploratória de uma série temporal

## Métodos familiares
- Propriedades estatísticas
    - Média
    - Variância
    - Desvio padrão
- Distribuição dos dados
- Correlação

## Métodos específicos
- Estacionariedade
- Auto-correlação
- Correlações espúrias
- Decomposição da série

## Métodos exploratórios familiares
### Propriedades estatísticas

In [267]:
df_air.describe()

,Qtd passageiros
count,144.000000
mean,280.298611
std,119.966317
min,104.000000
25%,180.000000
50%,265.500000
75%,360.500000
max,622.000000


Criando colunas dos mêses

In [268]:
df_air['dt_month'] = df_air['Período'].dt.month_name()

### Analisando a distribuição mensal

In [269]:
fig = px.histogram(
    df_air,
    x="dt_month",
    y="Qtd passageiros",
    hover_data=["Qtd passageiros"],
    labels={
        "sum of Qtd passageiros": "Carga total de passageiros (kg)",
        "dt_month": "Mês",
    },
    barmode="group",
)
fig.update_layout(
    barmode="group",
    showlegend=True,
    title={
        "text": "Quantidades de passageiros por mês",
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    },
    height=300,
    width=900,
    margin={
        'l': 20,
        'r': 20,
        't': 40,
        'b': 20
    },
)
# fig.update(layout_coloraxis_showscale=False)
#fig.update_xaxes(categoryorder="array", categoryarray=dayofweek_name)
fig.show()

### Visualizando a distribuição de dados

In [270]:
fig = px.histogram(
    df_air,
    x='Qtd passageiros',
    marginal='box',  # or violin, rug
    hover_data=['Qtd passageiros'],
    height=400,
    width=600,
    #category_orders={'Trecho': trechos_analise}
)
fig.show()

In [271]:
fig = px.ecdf(
    df_air,
    x='Qtd passageiros',
    marginal='rug',  # or violin, rug
    hover_data=['Qtd passageiros'],
    height=400,
    width=600,
    #category_orders={'Trecho': trechos_analise}
)
fig.show()

## Métodos exploratórios específicos de séries temporais
- Estacionariedade
- Auto-correlação
- Análise de componentes

### Estacionariedade 
- Suas propriedades estatísticas são estáveis em um intervalo de tempo
    - Variância
    - Média
    - Covariância
- A série reflete um sistema no seu estado estacionário
- Análise
    - Visual
    - Estatística
- Muitos modelos dependem da séries ser estacionária;


#### Análise visual
##### Média em função do tempo
- A média varia rapidamente em função do tempo
- Característica de uma série não estacionária

In [272]:
fig = px.line(
    df_air.set_index('Período')['Qtd passageiros'].rolling(12).mean().reset_index().sort_values(by='Período'),
    x='Período',
    y='Qtd passageiros',
    labels={
        'value': 'Média da carga total',
        'variable': 'Trecho',
        'Data_hora': 'Data/hora'
    },
)
fig.update_yaxes(showticklabels=True)
fig.update_layout(
    barmode="group",
    showlegend=True,
    title={
        'text': 'Análise da média das séries temporais',
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
    height=300,
    width=900,
    margin={
        'l': 20,
        'r': 20,
        't': 40,
        'b': 20
    },
)
fig.show()

##### Variância em função do tempo:
- A variância também varia rapidamente em função do tempo;
- Característica de uma série não estacionária.

In [273]:
fig = px.line(
    df_air.set_index('Período')['Qtd passageiros'].rolling(12).var().reset_index().sort_values(by='Período'),
    x='Período',
    y='Qtd passageiros',
    labels={
        'value': 'Média da carga total',
        'variable': 'Trecho',
        'Data_hora': 'Data/hora'
    },
)
fig.update_yaxes(showticklabels=True)
fig.update_layout(
    barmode="group",
    showlegend=True,
    title={
        'text': 'Análise da variância das séries temporais',
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
    height=300,
    width=900,
    margin={
        'l': 20,
        'r': 20,
        't': 40,
        'b': 20
    },
)
fig.show()

### Análise estatística
#### Teste de Dickey-Fuller aumentado (Augmented Dickey-Fuller test) e autocorrelação
- É um teste formal acerca de quão estacionária são essas séries
- Vamos estimar os períodos de autocorrelação para analisar o impacto de eventos periódicos nos valores da série
No teste de Dickey-Fuller aumentado verifica-se se a série é não-estacionária:
- p-valor menor que 0.05
- Estatística de Dickley-Fuller > valor crítico 1%

In [274]:
plot_adf(df_air.set_index('Período')['Qtd passageiros'], 'Qtd passageiros')

### Resultados da estacionarariedade

Essa série não é estacionária mas podemos fazer transformações para que ela se torne estacionária, como a diferenciação:

In [275]:
df_air['diff'] = df_air['Qtd passageiros'].diff()
df_air = df_air[['Período', 'Qtd passageiros', 'diff', 'dt_month']]
df_air.head()

,Período,Qtd passageiros,diff,dt_month
0,1949-01-01,112.0,NaN,January
1,1949-02-01,118.0,6.0,February
2,1949-03-01,132.0,14.0,March
3,1949-04-01,129.0,-3.0,April
4,1949-05-01,121.0,-8.0,May


In [276]:
plot_adf(df_air.set_index('Período')['diff'].dropna(), 'Qtd passageiros')

#### A série torna-se estacionária, mas quais características ela tem?

In [277]:
fig = px.histogram(
    df_air,
    x='diff',
    marginal='box',  # or violin, rug
    height=400,
    width=600,
    #category_orders={'Trecho': trechos_analise}
)
fig.show()

#### Sua nova estatística

In [278]:
df_air.describe()

,Qtd passageiros,diff
count,144.000000,143.000000
mean,280.298611,2.237762
std,119.966317,33.754282
min,104.000000,-101.000000
25%,180.000000,-16.000000
50%,265.500000,4.000000
75%,360.500000,22.500000
max,622.000000,87.000000


Não é incomum diferenciar uma série mais de uma vez para deixar-la estacionária

In [279]:
plot_adf(df_air.set_index('Período')['diff'].diff().dropna(), 'Qtd passageiros')

### Auto-correlação
- A auto-correlação descreve a relação linear entre valores defasados de uma série temporal
- A função de auto-correlação fornece os coeficientes da auto-correlação da série em relação a todas as sua possíveis defasagens

In [280]:
analise_serie(df_air.set_index('Período')['Qtd passageiros'], 'Qtd passageiros')

Analisando o gráfico da auto-correlação
- A queda suavel de valores positivos indica uma forte tendência
- O comportamento oscilatório indica uma forte sazonalidade
- O pico no índice de lag 12 indica que a sazonalidade é de 12 meses

Análise da auto-correlação parcial
- Remove efeitos harmônicos dos lags
- Fica mais evidente relacionar a sazonalidade

#### Podemos analisar a auto-correlação da série diferenciada?

In [281]:
analise_serie(df_air.set_index('Período')['diff'].dropna(), 'Diferenciação')

### Decomposição da série

Séries temporais podem ser decompostas em 3 séries componentes:
- Tendência
    - Padrão de aumento ou diminuição em longo prazo
- Cíclica
    - Aumentos ou diminuições que não demonstram um período fixo
- Sazonalidade
    - Padrão exibido quando a série é influênciada por fatores sazonais
- Ruído
    - Tudo aquilo que não conseguimos determinar

#### Componentes da série

In [282]:
series_decomposition(df_air.set_index('Período')['Qtd passageiros'], 'Qtd passageiros', periodo=12)

Modelos de decomposição:
- Nesse caso a magnitudade da sazonalidade varia com a tendência
- Isso indica que a relação entre as componentes é multiplicativa e não aditiva
- A transformação logarítimica torna uma relação multiplicativa numa relação aditiva:
$$y_t = T_t \times S_t \times R_t \rightarrow \log(y_t) = \log(T_t) + \log(S_t) + \log(R_t)$$

Modelos de decomposição:

In [283]:
series_decomposition(np.log(df_air.set_index('Período')['Qtd passageiros']), 'log(Qtd passageiros)', periodo=12)

# Modelagem
- Modelamento de séries temporáis para previsão é uma área de pesquisa da estatística;
- Assumimos que há relação entre a variável dimensionalizada e o momento no tempo;
- Relação entre a variável dimensionalizada e outras covariáveis.

## Machine learning

- Ensinar o computador a aprender a partir de dados
- Identificando padrões e relações nesses dados
- Predizer comportamentos, tendências para ajudar a tomar decisões


### Modelos
- Naive
    - last
    - mean
    - drift
- AutoETS
- ThetaForecaster
- Prophet

#### Pontuação do modelo
- Usamos o MAPE (*Mean absolute percentage error*)
- Medida percentual da acurácia de previsão do modelo.

In [284]:
y = df_air.set_index('Período')['Qtd passageiros'].to_period(freq='M')

In [285]:
forecasters = {
    'Naive last' : NaiveForecaster(strategy='last'),
    'Naive mean' : NaiveForecaster(strategy='mean'),
    'Naive drift' : NaiveForecaster(strategy='drift'),
    'AutoETS': AutoETS(auto=True, sp=12, n_jobs=-1),
    'Prophet': Prophet(seasonality_mode='multiplicative',
                       n_changepoints=int(len(y) / 12),
                       yearly_seasonality=True),
    'ThetaForecaster': ThetaForecaster(sp=12)
}

#### Validação cruzada de séries temporaris
- É um método sofisticado de divisão train/test
- Um grupo de pontos consecutivos é chamado de série de treino
- Seguido de outros pontos consecultivos chamados de séries de teste
- O desempenho do modelo é medido a partir da distribuição da do erro dessas séries
- Avaliação sobre uma origem de previsão contínua

In [286]:
init_lenght = 0
train_length = 24
drop_lenght = 0
test_lenght = 12
step_lenght = 12

total = init_lenght + train_length + drop_lenght + test_lenght
total

36

In [287]:
windows_idx = get_slidewindows_steps(init_lenght, train_length, drop_lenght, test_lenght, step_lenght, n_steps=10)

In [288]:

# Cria os subplots com os títulos exibindo a validação cruzada
fig = make_subplots(
    rows=2,
    cols=1,
    vertical_spacing=0.1,
    shared_xaxes=True,
    specs=[
        [{'type': 'xy'}],
        [{'type': 'xy'}]
    ],
    subplot_titles=[
        'Validação cruzada tradicional',
        'Validação cruzada de séries temporais'
    ],
    row_heights=[0.2, 0.8]
    
)

# Gráfico da validação cruzada tradicional
fig.add_trace(
    go.Scatter(
        x=df_air['Período'],
        y=np.full(
            shape=len(df_air['Período']),
            fill_value=0,
            dtype='int'
        ),
        mode='markers',
        marker=dict(
            color='rgba(255, 255, 255, 0.5)',
            size=6
        ),
        showlegend=False
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(
        x=df_air['Período'][:119],
        y=np.full(
            shape=len(df_air['Período'][:119]),
            fill_value=0,
            dtype='int'
        ),
        mode='markers',
        marker=dict(
            color='yellow',
            size=6
        ),
        showlegend=False
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(
        x=df_air['Período'][119:],
        y=np.full(
            shape=len(df_air['Período'][119:]),
            fill_value=0,
            dtype='int'
        ),
        mode='markers',
        marker=dict(
            color='blue',
            size=6
        ),
        showlegend=False
    ),
    row=1,
    col=1
)


# Gráfico da validação cruzada de séries temporais
for i, window in enumerate(windows_idx):
    
    fig.add_trace(
        go.Scatter(
            x=df_air['Período'],
            y=np.full(
                shape=len(df_air['Período']),
                fill_value=i,
                dtype='int'
            ),
            mode='markers',
            marker=dict(
                color='rgba(255, 255, 255, 0.5)',
                size=6
            ),
            showlegend=False
        ),
    row=2,
    col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_air['Período'].iloc[window['train']],
            y=np.full(
                shape=len(df_air['Período'].iloc[window['train']]),
                fill_value=i,
                dtype='int'
            ),
            mode='markers',
            marker=dict(
                color='yellow',
                size=6
            ),
            showlegend=False
        ),
    row=2,
    col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_air['Período'].iloc[window['test']],
            y=np.full(
                shape=len(df_air['Período'].iloc[window['train']]),
                fill_value=i,
                dtype='int'
            ),
            mode='markers',
            marker=dict(
                color='blue',
                size=6
            ),
            showlegend=False
        ),
    row=2,
    col=1
    )
    
layout = go.Layout(
    {
        'showlegend': False,
        "yaxis": {
            'tickmode' : 'linear',
            'tick0' : 0,
            'dtick' : 1
        },
        'height':400,
        'width':900,
        'margin': {
            'l': 20,
            'r': 20,
            't': 20,
            'b': 20
        },
    }
)

fig.update_layout(layout)

fig.show()

In [289]:
result = pd.DataFrame()

for i, windows in tqdm(enumerate(windows_idx)):
    y_train = y.iloc[windows['train']]
    y_test = y.iloc[windows['test']]
    
    for model in forecasters.keys():
    
        forecaster = forecasters[model]
        forecaster.fit(y_train)
    
        y_pred = forecaster.predict(fh=y_test.index)
    
        mape = MeanAbsolutePercentageError(symmetric=True)
    
        error = mape(y_test, y_pred)
    
        result = pd.concat(
            [
                result,
                pd.DataFrame(
                    {
                        i :{
                            'model': model,
                            'previsao': 'Base',
                            'y_train' : y_train,
                            'y_test' : y_test,
                            'y_pred' : y_pred,
                            'mape' : error
                        }
                    }
                ).T
            ], axis=0
        )

0it [00:00, ?it/s]15:19:53 - cmdstanpy - INFO - Chain [1] start processing
15:20:13 - cmdstanpy - INFO - Chain [1] done processing
1it [00:22, 22.88s/it]15:20:17 - cmdstanpy - INFO - Chain [1] start processing
15:20:37 - cmdstanpy - INFO - Chain [1] done processing
2it [00:46, 23.27s/it]15:20:39 - cmdstanpy - INFO - Chain [1] start processing
15:20:55 - cmdstanpy - INFO - Chain [1] done processing
3it [01:04, 21.08s/it]15:20:57 - cmdstanpy - INFO - Chain [1] start processing
15:21:13 - cmdstanpy - INFO - Chain [1] done processing
4it [01:22, 19.73s/it]15:21:16 - cmdstanpy - INFO - Chain [1] start processing
15:21:34 - cmdstanpy - INFO - Chain [1] done processing
5it [01:44, 20.39s/it]15:21:39 - cmdstanpy - INFO - Chain [1] start processing
15:22:01 - cmdstanpy - INFO - Chain [1] done processing
6it [02:10, 22.37s/it]15:22:04 - cmdstanpy - INFO - Chain [1] start processing
15:22:24 - cmdstanpy - INFO - Chain [1] done processing
7it [02:33, 22.74s/it]15:22:27 - cmdstanpy - INFO - Chain [

### Os modelos naive
- Previsão baseada em suposições ingênuas sobre tendências passadas continuando.

In [290]:
plot_models = ['Naive last', 'Naive mean', 'Naive drift']
plot_desempenho(result, plot_models, result['previsao'].unique().tolist())

#### Naive last
- Repete o último valor.

In [291]:
plot_previsao(y, result, 'Naive last', 'Base')

#### Naive mean
- Repete a média dos dados de treino.

In [292]:
plot_previsao(y, result, 'Naive mean', 'Base')

#### Naive drift
- Ajusta uma linha entre o primeiro e o último ponto da janela e extrapolá-la para o futuro.

In [293]:
plot_previsao(y, result, 'Naive drift', 'Base')

### AutoETS
- Família de modelos baseados no modelo *exponential smoothing*;
- ETS:
    - E: tipo de erro;
    - T: tipo de tendência;
    - S: tipo de sazonalidade.


In [294]:
plot_models = ['Naive last', 'Naive mean', 'Naive drift', 'AutoETS']
plot_desempenho(result, plot_models, result['previsao'].unique().tolist())

In [295]:
plot_previsao(y, result, 'AutoETS', 'Base')

### Prophet
- Algorítimo de previsão do facebook;
- Modelo aditivo que leva em conta tendência e sazonalidades não lineares.

In [296]:
plot_models = ['Naive last', 'Naive mean', 'Naive drift', 'AutoETS', 'Prophet']
plot_desempenho(result, plot_models, result['previsao'].unique().tolist())

In [297]:
plot_previsao(y, result, 'Prophet', 'Base')

### ThetaForecaster
- Variação do modelo de *exponential smoothing*;
- Conceito de modificação da curvatural local da série temporal.

In [298]:
plot_desempenho(result, result['model'].unique().tolist(), result['previsao'].unique().tolist())

In [299]:
plot_previsao(y, result, 'ThetaForecaster', 'Base')

# Gerando *features* para a série temporal

- Conceito de *feature engineering*;
- Processo de encapsular as características mais importantes dos dados de séries temporais;
- Valores numéricos ou etiquetas de categorias

In [300]:
df_air['Mês'] = df_air['Período'].dt.month
df_air['Ano'] = df_air['Período'].dt.year - df_air['Período'].min().year

In [301]:
df_air.head()

,Período,Qtd passageiros,diff,dt_month,Mês,Ano
0,1949-01-01,112.0,NaN,January,1,0
1,1949-02-01,118.0,6.0,February,2,0
2,1949-03-01,132.0,14.0,March,3,0
3,1949-04-01,129.0,-3.0,April,4,0
4,1949-05-01,121.0,-8.0,May,5,0


In [302]:
data = [
    go.Scatter(
        x=df_air['Período'],
        y=df_air[col],
        yaxis='y2',
        name=col,
    ) for col in ['Mês', 'Ano']
]

data.insert(
    0,
    go.Scatter(
        x=df_air['Período'],
        y=df_air['Qtd passageiros'],
        yaxis='y1',
        name='Série'
    )
)

layout = go.Layout(
    title='Covariantes temporais',
    yaxis=dict(title='Qtd de passageiros'),
    yaxis2=dict(
        title='Mês e ano',
        overlaying='y',
        side='right'
    ),
    height=300,
    width=900,
    margin={
        'l': 20,
        'r': 20,
        't': 40,
        'b': 20
    },
)

go.Figure(data=data, layout=layout)

In [303]:
y = df_air.set_index('Período')['Qtd passageiros'].to_period(freq='M')
X = df_air.set_index('Período')[['Mês', 'Ano']].to_period(freq='M')

X.head()

,Mês,Ano
Período,,
1949-01,1,0
1949-02,2,0
1949-03,3,0
1949-04,4,0
1949-05,5,0


In [304]:
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError

for i, windows in tqdm(enumerate(windows_idx)):
    y_train = y.iloc[windows['train']]
    y_test = y.iloc[windows['test']]
    X_train = X.iloc[windows['train']]
    X_test = X.iloc[windows['test']]
    
    for model in forecasters.keys():
    
        forecaster = forecasters[model]
        forecaster.fit(y_train, X_train)
    
        y_pred = forecaster.predict(fh=y_test.index, X=X_test)
    
        mape = MeanAbsolutePercentageError(symmetric=True)
    
        error = mape(y_test, y_pred)
    
        result = pd.concat(
            [
                result,
                pd.DataFrame(
                    {
                        i :{
                            'model': model,
                            'previsao': 'Time features',
                            'y_train' : y_train,
                            'y_test' : y_test,
                            'y_pred' : y_pred,
                            'mape' : error
                        }
                    }
                ).T
            ], axis=0
        )

0it [00:00, ?it/s]15:23:26 - cmdstanpy - INFO - Chain [1] start processing
15:23:44 - cmdstanpy - INFO - Chain [1] done processing
1it [00:19, 19.09s/it]15:23:46 - cmdstanpy - INFO - Chain [1] start processing
15:24:04 - cmdstanpy - INFO - Chain [1] done processing
2it [00:39, 19.80s/it]15:24:05 - cmdstanpy - INFO - Chain [1] start processing
15:24:21 - cmdstanpy - INFO - Chain [1] done processing
3it [00:56, 18.41s/it]15:24:22 - cmdstanpy - INFO - Chain [1] start processing
15:24:40 - cmdstanpy - INFO - Chain [1] done processing
4it [01:15, 18.80s/it]15:24:42 - cmdstanpy - INFO - Chain [1] start processing
15:25:00 - cmdstanpy - INFO - Chain [1] done processing
5it [01:35, 19.36s/it]15:25:02 - cmdstanpy - INFO - Chain [1] start processing
15:25:14 - cmdstanpy - INFO - Chain [1] done processing
6it [01:49, 17.36s/it]15:25:16 - cmdstanpy - INFO - Chain [1] start processing
15:25:39 - cmdstanpy - INFO - Chain [1] done processing
7it [02:15, 20.12s/it]15:25:42 - cmdstanpy - INFO - Chain [

## Desempenho dos modelos com as *features* temporais

In [305]:
plot_desempenho(result, result['model'].unique().tolist(), result['previsao'].unique().tolist())

## Importância do uso de *features* no modelo
- Não apenas no desempenho do modelo;
- Permite analisar a contribuição de algum evento nos dados da série temporal.
- Exemplos:
    - Clima;
    - Feriados;
    - Eventos esportivos;
    - Eventos culturais.

# Transformações e ajustes
- Transformar e ajustar os dados podem nos fornecer séries mais simples:
    - Eventos de calendário;
    - Ajustes populacionais.
- Transformações matemáticas:
    - Raíz;
    - Logarítimo.

In [306]:
log_y = np.log(y)

In [307]:
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError

for i, windows in tqdm(enumerate(windows_idx)):
    y_train = y.iloc[windows['train']]
    y_test = y.iloc[windows['test']]
    
    for model in forecasters.keys():
    
        forecaster = forecasters[model]
        forecaster.fit(np.log(y_train))
    
        y_pred = np.exp(forecaster.predict(fh=y_test.index))
        
        mape = MeanAbsolutePercentageError(symmetric=True)
        error = mape(y_test, y_pred)
    
        result = pd.concat(
            [
                result,
                pd.DataFrame(
                    {
                        i :{
                            'model': model,
                            'previsao': 'Log',
                            'y_train': y_train,
                            'y_test': y_test,
                            'y_pred': y_pred,
                            'mape': error
                        }
                    }
                ).T
            ], axis=0
        )

0it [00:00, ?it/s]15:26:47 - cmdstanpy - INFO - Chain [1] start processing
15:27:05 - cmdstanpy - INFO - Chain [1] done processing
1it [00:19, 19.14s/it]15:27:06 - cmdstanpy - INFO - Chain [1] start processing
15:27:23 - cmdstanpy - INFO - Chain [1] done processing
2it [00:37, 18.93s/it]15:27:25 - cmdstanpy - INFO - Chain [1] start processing
15:27:42 - cmdstanpy - INFO - Chain [1] done processing
3it [00:56, 18.96s/it]15:27:44 - cmdstanpy - INFO - Chain [1] start processing
15:28:02 - cmdstanpy - INFO - Chain [1] done processing
4it [01:16, 19.33s/it]15:28:03 - cmdstanpy - INFO - Chain [1] start processing
15:28:21 - cmdstanpy - INFO - Chain [1] done processing
5it [01:35, 19.14s/it]15:28:22 - cmdstanpy - INFO - Chain [1] start processing
15:28:45 - cmdstanpy - INFO - Chain [1] done processing
6it [01:59, 20.86s/it]15:28:47 - cmdstanpy - INFO - Chain [1] start processing
15:29:07 - cmdstanpy - INFO - Chain [1] done processing
7it [02:21, 21.11s/it]15:29:08 - cmdstanpy - INFO - Chain [

In [308]:
plot_desempenho(result, result['model'].unique().tolist(), result['previsao'].unique().tolist())

## Combinando transformações e features

In [309]:
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError

for i, windows in tqdm(enumerate(windows_idx)):
    y_train = y.iloc[windows['train']]
    y_test = y.iloc[windows['test']]
    X_train = X.iloc[windows['train']]
    X_test = X.iloc[windows['test']]
    
    for model in forecasters.keys():
    
        forecaster = forecasters[model]
        forecaster.fit(np.log(y_train), X_train)
    
        y_pred = np.exp(forecaster.predict(fh=y_test.index, X=X_test))
        
        mape = MeanAbsolutePercentageError(symmetric=True)
        error = mape(y_test, y_pred)
    
        result = pd.concat(
            [
                result,
                pd.DataFrame(
                    {
                        i :{
                            'model': model,
                            'previsao': 'Log + features',
                            'y_train' : y_train,
                            'y_test' : y_test,
                            'y_pred' : y_pred,
                            'mape' : error
                        }
                    }
                ).T
            ], axis=0
        )

0it [00:00, ?it/s]15:30:01 - cmdstanpy - INFO - Chain [1] start processing
15:30:18 - cmdstanpy - INFO - Chain [1] done processing
1it [00:17, 17.57s/it]15:30:19 - cmdstanpy - INFO - Chain [1] start processing
15:30:28 - cmdstanpy - INFO - Chain [1] done processing
2it [00:27, 13.29s/it]15:30:29 - cmdstanpy - INFO - Chain [1] start processing
15:30:46 - cmdstanpy - INFO - Chain [1] done processing
3it [00:46, 15.65s/it]15:30:48 - cmdstanpy - INFO - Chain [1] start processing
15:31:06 - cmdstanpy - INFO - Chain [1] done processing
4it [01:06, 17.38s/it]15:31:07 - cmdstanpy - INFO - Chain [1] start processing
15:31:26 - cmdstanpy - INFO - Chain [1] done processing
5it [01:26, 18.42s/it]15:31:28 - cmdstanpy - INFO - Chain [1] start processing
15:31:33 - cmdstanpy - INFO - Chain [1] done processing
6it [01:33, 14.47s/it]15:31:35 - cmdstanpy - INFO - Chain [1] start processing
15:31:54 - cmdstanpy - INFO - Chain [1] done processing
7it [01:54, 16.50s/it]15:31:56 - cmdstanpy - INFO - Chain [

In [310]:
plot_desempenho(result, result['model'].unique().tolist(), result['previsao'].unique().tolist())

# Visualizando os melhores ajustes de modelos

In [311]:
modelo = 'AutoETS'
previsao = 'Log + features'
plot_previsao(y, result, modelo, previsao)

In [312]:
modelo = 'AutoETS'
previsao = 'Log'
plot_previsao(y, result, modelo, previsao)

In [313]:
modelo = 'Prophet'
previsao = 'Base'
plot_previsao(y, result, modelo, previsao)

In [314]:
modelo = 'Prophet'
previsao = 'Time features'
plot_previsao(y, result, modelo, previsao)

# Conclusão
- O que são previsões:
    - Os parâmetros que garantem uma boa previsão;
- Séries temporáis:
    - Visualizações;
    - Métodos de análise familiares;
    - Métodos de análise exclusivos.
- Modelagem:
    - Machine learning;
    - Validação cruzada;
    - Listamos modelos;
    - Pontuação desses modelos;
    - Criação de *features*;
    - Transformações
- Execução:
    - Treinamento;
    - Avaliação;
    - Seleção.